In [18]:
import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [19]:
nlp = spacy.load('en_core_web_lg')

In [20]:
df = pd.read_csv('../Datasets/train/steam_ds.csv')

In [21]:
df.head()

,Unnamed: 0,review_id,title,year,user_review,user_suggestion
0,0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,Positive
1,1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",Positive
2,2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",Positive
3,3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",Positive
4,4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,Positive


In [23]:
df.drop('Unnamed:0', inplace=True)

KeyError: "['Unnamed:0'] not found in axis"

In [6]:
def convertToSentiment(value):
    if(value == 1):
        return 'Positive'
    else:
        return 'Negative'

In [7]:
df['user_suggestion'] = df['user_suggestion'].apply(convertToSentiment)

# Seperate data into train and test split

### Extract document and Target labels from the dataset

In [16]:
document = df['user_review']
sentiment = df['user_suggestion']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(document, sentiment, test_size=0.20, random_state=42)

In [ ]:
def convertDocumenttotfidf(doc):
    tfidf_vect = TfidfVectorizer()
    tfidf_vect_fit = tfidf_vect.fit(doc)
    tfiddfVector = vectorize(doc,tfidf_vect_fit)
    X_tfidf = tfidf_vect_fit.transform(data)